# Make CEDS and GCP historical emissions

- N2O 
  - FFI & Agriculture: PRIMAP-hist v2.4 1750 to 2021
  - input4mips for 1750 to 1996
  - GFEDv4.1s for 1997 to 2021
- CH4
  - FFI & Agriculture: PRIMAP-hist v2.4 1750 to 2021
  - input4mips for 1750 to 1996
  - GFEDv4.1s for 1997 to 2021
- CO2: Global carbon project 1750 to 2021 with 2022 estimate
- SLCFs: CEDS to 1750 to 2019 (STILL)

Biomass burning for SLCFs will come from GFED4.1s

- Do we need to include biomass burning N2O and CH4 emissions?

We also first need to fill in co-emittants for CH4 in order to get methane lifetime as a residual

Carbon dioxide
- 1750: 278.3 ppm
- 1850: 285.5 ppm

Methane
- 1750: 729.2 ppb
- 1850: 807.6 ppb

In [ ]:
import os
import zipfile

import numpy as np
import pandas as pd
import pooch

In [ ]:
update = pd.DataFrame(columns=range(1750, 2022))

## CO2

In [ ]:
df_co2 = pd.read_csv('../data/global_carbon_project/co2_emissions_gcp2022.csv', index_col='Year')
update.loc['CO2 FFI'] = df_co2['fossil emissions including carbonation']*44.009/12.011
update.loc['CO2 AFOLU'] = df_co2['land-use change emissions']*44.009/12.011

#deflate = df_co2['AFOLU'].loc[1850]/df_co2['AFOLU'].loc[1860]
#update.loc['CO2 AFOLU', 1750:1849] = deflate**np.arange(11, 1, -0.1) * df_co2['AFOLU'].loc[1860] * 44.009/12.011
update.loc['CO2 AFOLU', 1750:1850] = np.linspace(0, 1, 101) * df_co2['land-use change emissions'].loc[1850] * 44.009/12.011

update.loc['CO2 AFOLU'].plot()

In [ ]:
update

In [ ]:
# CEDS emissions: newest version
ceds_zip = pooch.retrieve(
    "https://zenodo.org/record/4741285/files/CEDS_v2021-04-21_emissions.zip",
    known_hash="md5:01659c651754a66ddf3d79715a2ba841",
    progressbar=True
)

In [ ]:
os.makedirs('../data/ceds/CEDS_v2021-04-21_emissions/', exist_ok=True)
with zipfile.ZipFile(ceds_zip, 'r') as z:
    z.extractall(path='../data/ceds/CEDS_v2021-04-21_emissions/')

## CH4: anthro from PRIMAP-hist, also need biomass from RCMIP

In [ ]:
rcmip_emissions_file = pooch.retrieve(
    url="doi:10.5281/zenodo.4589756/rcmip-emissions-annual-means-v5-1-0.csv",
    known_hash="md5:4044106f55ca65b094670e7577eaf9b3",
)

In [ ]:
emis_df = pd.read_csv(rcmip_emissions_file)

In [ ]:
# PRIMAP-hist emissions: newest version
primap24 = pooch.retrieve(
    "https://zenodo.org/record/7179775/files/Guetschow-et-al-2022-PRIMAP-hist_v2.4_no_rounding_11-Oct-2022.csv",
    known_hash="md5:3a40d9abb47fe688295af36301424679",
    progressbar=True
)
primap24_df = pd.read_csv(primap24)

In [ ]:
primap24_df.loc[
    (primap24_df['scenario (PRIMAP-hist)']=='HISTTP')&
    (primap24_df['entity']=='CH4')&
    (primap24_df['category (IPCC2006_PRIMAP)']=='M.0.EL')&
    (primap24_df['area (ISO3)']=='EARTH')
]

In [ ]:
primap24_df.loc[
    (primap24_df['scenario (PRIMAP-hist)']=='HISTTP')&
    (primap24_df['entity']=='CH4')&
    (primap24_df['category (IPCC2006_PRIMAP)']=='M.0.EL')&
    (primap24_df['area (ISO3)']=='EARTH'),
'1750':].T.plot()

In [ ]:
update

In [ ]:
ch4 = primap24_df.loc[
    (primap24_df['scenario (PRIMAP-hist)']=='HISTTP')&
    (primap24_df['entity']=='CH4')&
    (primap24_df['category (IPCC2006_PRIMAP)']=='M.0.EL')&
    (primap24_df['area (ISO3)']=='EARTH'),
'1750':].values.squeeze()

In [ ]:
# from updated GFED
gfed41s = pd.read_csv('../data/gfed4.1s/gfed4.1s_1997-2021.csv', index_col=0)
gfed41s.loc['N2O', '1997':'2021'].values

In [ ]:
rcmip = emis_df.loc[
    (emis_df['Scenario']=='ssp245')&
    (emis_df['Region']=='World')&
    (emis_df['Variable'].str.startswith(f'Emissions|CH4|')),
:]
ceds_rcmip = [f'Emissions|CH4|MAGICC AFOLU|Agriculture', f'Emissions|CH4|MAGICC Fossil and Industrial']
uva_rcmip = [
    f'Emissions|CH4|MAGICC AFOLU|Agricultural Waste Burning', 
    f'Emissions|CH4|MAGICC AFOLU|Forest Burning', 
    f'Emissions|CH4|MAGICC AFOLU|Grassland Burning', 
    f'Emissions|CH4|MAGICC AFOLU|Peat Burning'
]
update.loc['CH4', 1750:1996] = (
    ch4[:247]/1000 +
    rcmip.loc[rcmip['Variable'].isin(uva_rcmip), '1750':'1996'].interpolate(axis=1).sum().values.squeeze()#[:-4]
)
update.loc['CH4', 1997:2021] = (
    ch4[247:]/1000 +
    gfed41s.loc['CH4', '1997':'2021'].values.squeeze()
)

In [ ]:
update.T.plot()

## N2O - from PRIMAP-hist

In [ ]:
n2o = primap24_df.loc[
    (primap24_df['scenario (PRIMAP-hist)']=='HISTTP')&
    (primap24_df['entity']=='N2O')&
    (primap24_df['category (IPCC2006_PRIMAP)']=='M.0.EL')&
    (primap24_df['area (ISO3)']=='EARTH'),
'1750':].values.squeeze()

In [ ]:
# from AR6
n2o_biomass = pd.read_csv('../data/vua/n2o_biomass_burning_global_totals_1750-2015.csv', index_col='year')
n2o_biomass.loc[1750:2015]

In [ ]:
update.loc['N2O', 1750:1996] = (
    n2o[:247]/1000 +
    n2o_biomass.loc[1750:1996].values.squeeze()
)
update.loc['N2O', 1997:2021] = (
    n2o[247:]/1000 +
    gfed41s.loc['N2O', '1997':'2021'].values.squeeze()
)

In [ ]:
update

In [ ]:
update.loc['N2O'].T.plot()

## SLCFs = CEDS + VUA

In [ ]:
gfed41s.loc['NMVOC'] = pd.concat((gfed41s.loc['C2H6':'C3H6O'], gfed41s.loc['C2H6S':])).sum()
gfed41s

In [ ]:
species = ['Sulfur', 'CO', 'VOC', 'NOx', 'BC', 'OC', 'NH3']

unit_convert = {specie: 1/1000 for specie in species}
ceds_names = {specie: specie for specie in species}
ceds_names.update({'VOC': 'NMVOC', 'Sulfur': 'SO2'})


for specie in species:
    df_ceds_latest = pd.read_csv(f'../data/ceds/CEDS_v2021-04-21_emissions/{ceds_names[specie]}_global_CEDS_emissions_by_sector_2021_04_21.csv')
    df_ceds_latest.drop(columns=['em', 'sector', 'units'], inplace=True)
    rcmip = emis_df.loc[
        (emis_df['Scenario']=='ssp245')&
        (emis_df['Region']=='World')&
        (emis_df['Variable'].str.startswith(f'Emissions|{specie}|')),
    :]
    ceds_rcmip = [f'Emissions|{specie}|MAGICC AFOLU|Agriculture', f'Emissions|{specie}|MAGICC Fossil and Industrial']
    uva_rcmip = [
        f'Emissions|{specie}|MAGICC AFOLU|Agricultural Waste Burning', 
        f'Emissions|{specie}|MAGICC AFOLU|Forest Burning', 
        f'Emissions|{specie}|MAGICC AFOLU|Grassland Burning', 
        f'Emissions|{specie}|MAGICC AFOLU|Peat Burning'
    ]
    update.loc[specie, 1750:1996] = (
        df_ceds_latest.sum().values[:247] * unit_convert[specie] +
        rcmip.loc[rcmip['Variable'].isin(uva_rcmip), '1750':'1996'].interpolate(axis=1).sum().values.squeeze()
    )
    update.loc[specie, 1997:2019] = (
        df_ceds_latest.sum().values[247:] * unit_convert[specie] +
        gfed41s.loc[ceds_names[specie], '1997':'2019'].values.squeeze()
    )
    update.loc[specie, 2020:2021] = (
        df_ceds_latest.sum().values[-1] * unit_convert[specie] +
        gfed41s.loc[ceds_names[specie], '2020':'2021'].values.squeeze()
    )

In [ ]:
update.T.plot()

In [ ]:
emis_df.loc[
        (emis_df['Scenario']=='ssp245')&
        (emis_df['Region']=='World')&
        (emis_df['Variable'].str.startswith(f'Emissions|CO2|MAGICC AFOLU')),
    '1750':'2020'].T.plot()

In [ ]:
update.loc['CO2 AFOLU', :].plot()

In [ ]:
update

In [ ]:
os.makedirs('../data/emissions/', exist_ok=True)
update.to_csv('../data/emissions/co2_ch4_n2o_slcfs_1750-2021.csv')